In [ ]:
# import required libraries
# requests.auth to securely pass API key with the request
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import time

In [ ]:
# Companies House API key
API_KEY = ''

# API endpoint for advanced company search
SEARCH_URL = 'https://api.company-information.service.gov.uk/advanced-search/companies'

In [ ]:
# Load Excel file containing companies with missing SIC codes
df = pd.read_excel('Dataset/Companies Missing SIC code.xlsx')

In [ ]:
# Add empty columns to store fetched Company Numbers and SIC Codes
df['CompanyNumber'] = ''
df['SIC_Codes'] = ''

In [ ]:
# Function to fetch company number and SIC codes using company name
def get_company_details(company_name):
    params = {
        'company_name_includes': company_name, # Use the company name as the search query
        'size': 1  # Limit to top 1 match for accuracy
    }
    try:
        response = requests.get(SEARCH_URL, params=params, auth=HTTPBasicAuth(API_KEY, ''))
        if response.status_code == 200:
            data = response.json()
            # Extract 'items' from the JSON (the list of matching companies)
            items = data.get('items', [])
            if items:
                item = items[0]
                return item.get('company_number', ''), item.get('sic_codes', [])
    except Exception as e:
        print(f"Error fetching {company_name}: {e}")
    return '', []

In [ ]:
# Loop through each company to fetch and fill missing data
for idx, row in df.iterrows():
    # Extract the company name from the current row in the 'CompanyName' column
    company = row['CompanyName']
    number, sic = get_company_details(company)
    df.at[idx, 'CompanyNumber'] = number
    df.at[idx, 'SIC_Codes'] = ', '.join(sic) if sic else ''
    print(f"{company} → {number} | {sic}")

    time.sleep(0.5)  # Sleep to prevent hitting rate limits

In [ ]:
# Save the updated DataFrame to a new Excel file
df.to_excel('Dataset/Companies_With_SIC_Filled.xlsx', index=False)
print("Done!")